# 🏪 コンビニ店長向け実践ダッシュボード - 売上最大化システム

## 📋 このダッシュボードでできること

### 1️⃣ **毎日の意思決定支援**
- ✅ 今日の売上予測と発注量の最適化
- ✅ 他店との比較で自店舗の課題を発見
- ✅ 異常値の自動検知とアラート

### 2️⃣ **具体的な改善施策**
- ✅ 売上向上につながる商品・時間帯の特定
- ✅ 気象条件に応じた在庫調整
- ✅ 曜日・イベント別の販売戦略

### 3️⃣ **データドリブン経営**
- ✅ 前年同日比較で成長を可視化
- ✅ トレンド分析で将来予測
- ✅ ABC分析で重点商品を特定

---

## 🎯 店長が知りたい5つの質問

1. **今日の売上は予測通りか？**
2. **他店と比べて自店舗はどうか？**
3. **どの商品を重点的に発注すべきか？**
4. **どんな改善施策が効果的か？**
5. **明日・来週の売上はどうなるか？**

In [ ]:
# 日本語フォント設定（共通モジュール）
import warnings
warnings.filterwarnings('ignore')

# よく使うライブラリを先に読み込む
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ウィジェットの有無を通知・フラグ化
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print('✅ ipywidgets利用可能')
except Exception:
    WIDGETS_AVAILABLE = False
    print('⚠️ ipywidgets未インストール - 一部機能制限')

import font_setup
JP_FP = font_setup.setup_fonts(show_test=False)


---

# 📊 セクション1: 今日の状況把握（デイリーレポート）

## 店長が最初に確認すべき3つの指標

In [ ]:
# 🎯 店舗選択（あなたの店舗を選んでください）
MY_STORE = df['店舗'].unique()[0]  # 最初の店舗をデフォルト
print(f"\n🏪 選択店舗: {MY_STORE}")

# 利用可能な店舗一覧
print("\n📋 利用可能な店舗:")
for i, store in enumerate(df['店舗'].unique(), 1):
    print(f"   {i}. {store}")

# 自店舗データ抽出
my_store_df = df[df['店舗'] == MY_STORE].copy()

# 日次集計
daily_sales = my_store_df.groupby('日付').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '商品名': 'nunique',
    '昨年同日_売上': 'first'
}).reset_index()

daily_sales.columns = ['日付', '売上金額', '売上数量', '販売商品数', '昨年同日_売上']
daily_sales['客単価'] = daily_sales['売上金額'] / daily_sales['売上数量']
daily_sales['前年比'] = (daily_sales['売上金額'] / daily_sales['昨年同日_売上'] - 1) * 100

# 最新日のサマリー
latest_date = daily_sales['日付'].max()
latest_data = daily_sales[daily_sales['日付'] == latest_date].iloc[0]

print(f"\n📅 最新日: {latest_date.strftime('%Y-%m-%d (%a)')}")
print("=" * 50)
print(f"💰 売上金額:     ¥{latest_data['売上金額']:,.0f}")
print(f"📦 売上数量:     {latest_data['売上数量']:.0f}個")
print(f"💳 客単価:       ¥{latest_data['客単価']:.0f}")
print(f"📈 前年比:       {latest_data['前年比']:+.1f}%")
print(f"🎯 販売商品数:   {latest_data['販売商品数']:.0f}種類")

# 前年比の評価
if latest_data['前年比'] > 5:
    print("\n✨ 素晴らしい！前年を大きく上回っています")
elif latest_data['前年比'] > 0:
    print("\n✅ 順調です。前年を上回っています")
elif latest_data['前年比'] > -5:
    print("\n⚠️ 注意：前年をやや下回っています")
else:
    print("\n🚨 要改善：前年を大きく下回っています")

In [ ]:
# 📈 売上トレンドの可視化（直近30日）
recent_days = 30
recent_sales = daily_sales.tail(recent_days)

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. 売上金額の推移
ax1 = axes[0, 0]
ax1.plot(recent_sales['日付'], recent_sales['売上金額'], 
         marker='o', linewidth=2, label='今年', color='#2E86AB')
ax1.plot(recent_sales['日付'], recent_sales['昨年同日_売上'], 
         marker='s', linewidth=2, linestyle='--', label='昨年', color='#A23B72', alpha=0.7)
ax1.set_title('Daily Sales Trend (Last 30 Days, fontproperties=JP_FP)', fontsize=14)
ax1.set_ylabel('Sales Amount (JPY, fontproperties=JP_FP)', fontsize=12)
ax1.legend(prop=JP_FP)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# 2. 前年比の推移
ax2 = axes[0, 1]
colors = ['green' if x > 0 else 'red' for x in recent_sales['前年比']]
ax2.bar(recent_sales['日付'], recent_sales['前年比'], color=colors, alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax2.set_title('YoY Growth Rate (%, fontproperties=JP_FP)', fontsize=14)
ax2.set_ylabel('Growth Rate (%, fontproperties=JP_FP)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# 3. 客単価の推移
ax3 = axes[1, 0]
ax3.plot(recent_sales['日付'], recent_sales['客単価'], 
         marker='o', linewidth=2, color='#F18F01')
ax3.axhline(y=recent_sales['客単価'].mean(), 
            color='red', linestyle='--', label='Average')
ax3.set_title('Customer Spend per Transaction', fontsize=14, fontproperties=JP_FP)
ax3.set_ylabel('Amount per Customer (JPY, fontproperties=JP_FP)', fontsize=12)
ax3.legend(prop=JP_FP)
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# 4. 売上数量の推移
ax4 = axes[1, 1]
ax4.plot(recent_sales['日付'], recent_sales['売上数量'], 
         marker='o', linewidth=2, color='#6A4C93')
ax4.set_title('Sales Volume Trend', fontsize=14, fontproperties=JP_FP)
ax4.set_ylabel('Quantity Sold', fontsize=12, fontproperties=JP_FP)
ax4.grid(True, alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 📊 サマリー統計
print("\n📊 直近30日間のサマリー")
print("=" * 50)
print(f"平均日商:     ¥{recent_sales['売上金額'].mean():,.0f}")
print(f"最高日商:     ¥{recent_sales['売上金額'].max():,.0f}")
print(f"最低日商:     ¥{recent_sales['売上金額'].min():,.0f}")
print(f"平均前年比:   {recent_sales['前年比'].mean():+.1f}%")
print(f"平均客単価:   ¥{recent_sales['客単価'].mean():.0f}")

---

# 🏆 セクション2: 他店との比較（ベンチマーク分析）

## あなたの店舗は他店と比べてどうか？

In [ ]:
# 店舗別の日次集計
store_daily = df.groupby(['店舗', '日付']).agg({
    '売上金額': 'sum',
    '売上数量': 'sum'
}).reset_index()

store_daily['客単価'] = store_daily['売上金額'] / store_daily['売上数量']

# 店舗別サマリー（直近30日平均）
recent_period = store_daily['日付'].max() - pd.Timedelta(days=30)
store_summary = store_daily[store_daily['日付'] >= recent_period].groupby('店舗').agg({
    '売上金額': 'mean',
    '売上数量': 'mean',
    '客単価': 'mean'
}).round(0)

store_summary.columns = ['平均日商', '平均売上数量', '平均客単価']
store_summary['自店舗'] = store_summary.index == MY_STORE

# ランキング
store_summary['売上ランク'] = store_summary['平均日商'].rank(ascending=False).astype(int)
store_summary = store_summary.sort_values('平均日商', ascending=False)

print("\n🏆 店舗別パフォーマンス比較（直近30日平均）")
print("=" * 80)
print(f"{'店舗名':<30} {'平均日商':>15} {'平均数量':>12} {'客単価':>10} {'ランク':>6}")
print("=" * 80)

for idx, row in store_summary.iterrows():
    marker = "★" if row['自店舗'] else " "
    print(f"{marker} {idx:<28} ¥{row['平均日商']:>13,.0f} {row['平均売上数量']:>11,.0f} ¥{row['平均客単価']:>8,.0f} {row['売上ランク']:>5}位")

# 自店舗の位置
my_rank = store_summary[store_summary['自店舗']]['売上ランク'].values[0]
total_stores = len(store_summary)

print("\n📍 あなたの店舗の位置づけ")
print("=" * 50)
print(f"全{total_stores}店舗中 {my_rank}位")

if my_rank == 1:
    print("🥇 トップ店舗です！この調子で！")
elif my_rank <= total_stores * 0.3:
    print("🥈 上位店舗です。トップを目指しましょう！")
elif my_rank <= total_stores * 0.7:
    print("📊 中位店舗です。上位店舗の施策を参考に！")
else:
    print("⚠️ 下位店舗です。改善の余地が大きいです！")

In [ ]:
# 📊 店舗比較の可視化
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 色設定（自店舗を強調）
colors = ['#FF6B6B' if x else '#4ECDC4' for x in store_summary['自店舗']]

# 1. 平均日商比較
ax1 = axes[0]
store_summary['平均日商'].plot(kind='barh', ax=ax1, color=colors)
ax1.set_title('Average Daily Sales Comparison', fontsize=14, fontproperties=JP_FP)
ax1.set_xlabel('Average Daily Sales (JPY, fontproperties=JP_FP)', fontsize=12)
ax1.grid(axis='x', alpha=0.3)

# 2. 平均数量比較
ax2 = axes[1]
store_summary['平均売上数量'].plot(kind='barh', ax=ax2, color=colors)
ax2.set_title('Average Quantity Sold Comparison', fontsize=14, fontproperties=JP_FP)
ax2.set_xlabel('Average Quantity Sold', fontsize=12, fontproperties=JP_FP)
ax2.grid(axis='x', alpha=0.3)

# 3. 客単価比較
ax3 = axes[2]
store_summary['平均客単価'].plot(kind='barh', ax=ax3, color=colors)
ax3.set_title('Average Spend per Customer Comparison', fontsize=14, fontproperties=JP_FP)
ax3.set_xlabel('Avg Spend per Customer (JPY, fontproperties=JP_FP)', fontsize=12)
ax3.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# ギャップ分析
top_store_sales = store_summary['平均日商'].max()
my_store_sales = store_summary[store_summary['自店舗']]['平均日商'].values[0]
gap = top_store_sales - my_store_sales
gap_pct = (gap / top_store_sales) * 100

if gap > 0:
    print(f"\n💡 トップ店舗とのギャップ分析")
    print("=" * 50)
    print(f"ギャップ額:    ¥{gap:,.0f}/日")
    print(f"ギャップ率:    {gap_pct:.1f}%")
    print(f"月間換算:      ¥{gap * 30:,.0f}")
    print(f"年間換算:      ¥{gap * 365:,.0f}")

---

# 🎯 セクション3: 特徴量重要度分析（PyCaret）

## どの要因が売上に最も影響するか？

In [ ]:
# PyCaretで売上予測モデルを構築し、特徴量重要度を分析
print("🤖 機械学習モデルで売上要因を分析中...")
print("   (数分かかる場合があります)\n")

try:
    from pycaret.regression import *
    
    # モデリング用データ準備
    modeling_df = my_store_df.copy()
    
    # 目的変数: 売上金額
    # 説明変数: 利用可能な特徴量
    feature_cols = [
        '曜日', '週番号', '月', '日',
        '祝日フラグ', '週末フラグ', '連休フラグ',
        'GW', '盆休み', '給料日', '月初', '月末',
        '季節変動指数_月', '季節変動指数_週', '客数指数',
        '昨年同日_売上', '昨年同日_客数', '昨年同日_客単価'
    ]
    
    # 利用可能な列のみ選択
    available_features = [col for col in feature_cols if col in modeling_df.columns]
    
    # データ集計（日次）
    daily_modeling = modeling_df.groupby('日付').agg({
        '売上金額': 'sum',
        **{col: 'first' for col in available_features}
    }).reset_index()
    
    # 欠損値削除
    daily_modeling = daily_modeling.dropna(subset=['売上金額'] + available_features)
    
    if len(daily_modeling) >= 30:  # 最低30日分のデータが必要
        # PyCaret環境セットアップ
        setup_result = setup(
            data=daily_modeling,
            target='売上金額',
            session_id=123,
            verbose=False,
            html=False
        )
        
        # ベストモデルの選択
        best_model = compare_models(n_select=1, verbose=False)
        
        # モデルの詳細
        print("\n✅ モデル構築完了")
        print(f"   使用モデル: {type(best_model).__name__}")
        
        # 特徴量重要度の取得
        try:
            if hasattr(best_model, 'feature_importances_'):
                importance = pd.DataFrame({
                    '特徴量': available_features,
                    '重要度': best_model.feature_importances_
                }).sort_values('重要度', ascending=False)
                
                print("\n🔍 売上に最も影響する要因 TOP 10")
                print("=" * 60)
                for idx, row in importance.head(10).iterrows():
                    bar = "█" * int(row['重要度'] * 50)
                    print(f"{row['特徴量']:<25} {bar} {row['重要度']:.3f}")
                
                # グラフ化
                fig, ax = plt.subplots(figsize=(12, 6))
                importance.head(15).plot(x='特徴量', y='重要度', kind='barh', ax=ax, color='#FF6B6B')
                ax.set_title('Feature Importance for Sales Prediction', fontsize=14, fontproperties=JP_FP)
                ax.set_xlabel('Importance Score', fontsize=12, fontproperties=JP_FP)
                ax.set_ylabel('', fontproperties=JP_FP)
                plt.tight_layout()
                plt.show()
        except:
            print("⚠️ 特徴量重要度の抽出に失敗しました")
    else:
        print(f"⚠️ データ不足: {len(daily_modeling)}日分（最低30日必要）")
        
except ImportError:
    print("⚠️ PyCaretがインストールされていません")
    print("   'pip install pycaret' でインストールしてください")
except Exception as e:
    print(f"⚠️ エラー: {str(e)}")

---

# 💡 セクション4: 実践的な改善施策レコメンデーション

## データから導かれる具体的なアクション

In [ ]:
# 曜日別分析
weekday_analysis = my_store_df.groupby('曜日').agg({
    '売上金額': 'sum',
    '売上数量': 'sum'
}).reset_index()

weekday_analysis['客単価'] = weekday_analysis['売上金額'] / weekday_analysis['売上数量']
weekday_analysis['曜日名'] = weekday_analysis['曜日'].map({
    0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'
})

# 最高売上曜日と最低売上曜日
best_day = weekday_analysis.loc[weekday_analysis['売上金額'].idxmax()]
worst_day = weekday_analysis.loc[weekday_analysis['売上金額'].idxmin()]

print("\n📅 曜日別パフォーマンス分析")
print("=" * 60)
print(f"最高売上曜日: {best_day['曜日名']} (¥{best_day['売上金額']:,.0f})")
print(f"最低売上曜日: {worst_day['曜日名']} (¥{worst_day['売上金額']:,.0f})")
print(f"曜日間ギャップ: ¥{best_day['売上金額'] - worst_day['売上金額']:,.0f}")

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 曜日別売上
ax1 = axes[0]
weekday_analysis.plot(x='曜日名', y='売上金額', kind='bar', ax=ax1, color='#4ECDC4')
ax1.set_title('Sales by Day of Week', fontsize=14, fontproperties=JP_FP)
ax1.set_ylabel('Total Sales (JPY, fontproperties=JP_FP)', fontsize=12)
ax1.set_xlabel('', fontproperties=JP_FP)
ax1.tick_params(axis='x', rotation=0)
ax1.grid(axis='y', alpha=0.3)

# 曜日別客単価
ax2 = axes[1]
weekday_analysis.plot(x='曜日名', y='客単価', kind='bar', ax=ax2, color='#FF6B6B')
ax2.set_title('Avg Spend per Customer by Day', fontsize=14, fontproperties=JP_FP)
ax2.set_ylabel('Avg Spend (JPY, fontproperties=JP_FP)', fontsize=12)
ax2.set_xlabel('', fontproperties=JP_FP)
ax2.tick_params(axis='x', rotation=0)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 🎯 改善施策レコメンデーション
print("\n\n💡 データに基づく改善施策レコメンデーション")
print("=" * 70)

recommendations = []

# 1. 曜日別施策
if best_day['売上金額'] > worst_day['売上金額'] * 1.3:
    recommendations.append({
        '優先度': '高',
        'カテゴリ': '曜日別戦略',
        '施策': f"{worst_day['曜日名']}の売上を{best_day['曜日名']}レベルに近づける",
        '期待効果': f"¥{(best_day['売上金額'] - worst_day['売上金額']) * 4:,.0f}/月",
        '具体策': f"- {best_day['曜日名']}の売れ筋商品を{worst_day['曜日名']}にも強化\n" +
                  f"- {worst_day['曜日名']}限定セールの実施\n" +
                  f"- {worst_day['曜日名']}の客層に合わせた品揃え変更"
    })

# 2. 前年比改善施策
if daily_sales['前年比'].mean() < 0:
    recommendations.append({
        '優先度': '高',
        'カテゴリ': '前年比改善',
        '施策': '前年割れの要因分析と対策実施',
        '期待効果': f"¥{abs(daily_sales['前年比'].mean() * daily_sales['売上金額'].mean() / 100) * 30:,.0f}/月",
        '具体策': "- 前年の売れ筋商品の在庫確保\n" +
                  "- 新商品の積極的な陳列\n" +
                  "- 客数増加キャンペーンの実施"
    })

# 3. 客単価向上施策
avg_customer_spend = daily_sales['客単価'].mean()
if avg_customer_spend < 500:  # 目標客単価500円
    recommendations.append({
        '優先度': '中',
        'カテゴリ': '客単価向上',
        '施策': '客単価を¥500以上に引き上げ',
        '期待効果': f"¥{(500 - avg_customer_spend) * daily_sales['売上数量'].mean() * 30:,.0f}/月",
        '具体策': "- セット販売・まとめ買いの提案\n" +
                  "- 高単価商品の推奨販売\n" +
                  "- レジ前商品の強化"
    })

# 4. 他店ギャップ改善
if gap > 0 and gap > my_store_sales * 0.1:
    recommendations.append({
        '優先度': '高',
        'カテゴリ': '他店キャッチアップ',
        '施策': 'トップ店舗のベストプラクティス導入',
        '期待効果': f"¥{gap * 30:,.0f}/月",
        '具体策': "- トップ店舗の商品構成を参考\n" +
                  "- トップ店舗の販促施策を模倣\n" +
                  "- 店舗間での情報共有強化"
    })

# レコメンデーション表示
for i, rec in enumerate(recommendations, 1):
    print(f"\n【施策 {i}】 {rec['施策']}")
    print(f"優先度: {rec['優先度']} | カテゴリ: {rec['カテゴリ']}")
    print(f"期待効果: {rec['期待効果']}")
    print(f"\n具体的なアクション:")
    print(rec['具体策'])
    print("-" * 70)

# 総合効果
print("\n💰 施策実施による期待効果（合計）")
print("=" * 70)
print("すべての施策を実施した場合の売上向上見込み:")
print("※個別効果の単純合計ではなく、相乗効果も期待できます")

---

# 📦 セクション5: ABC分析 - どの商品に注力すべきか？

## パレートの法則: 20%の商品が80%の売上を生む

In [ ]:
# 商品別売上集計
product_sales = my_store_df.groupby('商品名').agg({
    '売上金額': 'sum',
    '売上数量': 'sum'
}).reset_index()

product_sales = product_sales.sort_values('売上金額', ascending=False)
product_sales['累積売上'] = product_sales['売上金額'].cumsum()
product_sales['累積売上比率'] = product_sales['累積売上'] / product_sales['売上金額'].sum() * 100

# ABC分類
def classify_abc(cumsum_ratio):
    if cumsum_ratio <= 70:
        return 'A'
    elif cumsum_ratio <= 90:
        return 'B'
    else:
        return 'C'

product_sales['ABC分類'] = product_sales['累積売上比率'].apply(classify_abc)

# ABC別サマリー
abc_summary = product_sales.groupby('ABC分類').agg({
    '商品名': 'count',
    '売上金額': 'sum',
    '売上数量': 'sum'
}).reset_index()

abc_summary.columns = ['ABC分類', '商品数', '売上金額', '売上数量']
abc_summary['売上構成比'] = abc_summary['売上金額'] / abc_summary['売上金額'].sum() * 100
abc_summary['商品数構成比'] = abc_summary['商品数'] / abc_summary['商品数'].sum() * 100

print("\n📊 ABC分析サマリー")
print("=" * 80)
print(f"{'分類':<10} {'商品数':>10} {'商品構成比':>12} {'売上金額':>15} {'売上構成比':>12}")
print("=" * 80)

for _, row in abc_summary.iterrows():
    print(f"{row['ABC分類']:<10} {row['商品数']:>10} {row['商品数構成比']:>11.1f}% "
          f"¥{row['売上金額']:>13,.0f} {row['売上構成比']:>11.1f}%")

print("\n💡 ABC分析の解釈")
print("=" * 80)
print("Aランク: 重点管理商品 - 欠品厳禁、常に在庫確保")
print("Bランク: 準重点商品 - 適正在庫を維持")
print("Cランク: 低回転商品 - 過剰在庫に注意、品揃えの見直し検討")

In [ ]:
# ABC分析の可視化
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. パレート図
ax1 = axes[0]
x = range(min(50, len(product_sales)))
ax1.bar(x, product_sales['売上金額'].head(50), color='#4ECDC4', alpha=0.7)
ax1_twin = ax1.twinx()
ax1_twin.plot(x, product_sales['累積売上比率'].head(50), 
              color='#FF6B6B', marker='o', linewidth=2)
ax1_twin.axhline(y=70, color='red', linestyle='--', label='70% Line')
ax1_twin.axhline(y=90, color='orange', linestyle='--', label='90% Line')
ax1.set_title('Pareto Chart (Top 50 Products, fontproperties=JP_FP)', fontsize=14)
ax1.set_xlabel('Product Rank', fontsize=12, fontproperties=JP_FP)
ax1.set_ylabel('Sales (JPY, fontproperties=JP_FP)', fontsize=12)
ax1_twin.set_ylabel('Cumulative %', fontsize=12, fontproperties=JP_FP)
ax1_twin.legend(prop=JP_FP)
ax1.grid(axis='y', alpha=0.3)

# 2. ABC分類の円グラフ
ax2 = axes[1]
colors_abc = ['#FF6B6B', '#FFD93D', '#6BCF7F']
ax2.pie(abc_summary['売上金額'], labels=abc_summary['ABC分類'], 
        autopct='%1.1f%%', colors=colors_abc, startangle=90)
ax2.set_title('Sales Distribution by ABC Category', fontsize=14, fontproperties=JP_FP)

plt.tight_layout()
plt.show()

# トップ10商品
print("\n🏆 売上TOP10商品（重点管理商品）")
print("=" * 90)
print(f"{'順位':<6} {'商品名':<40} {'売上金額':>15} {'売上数量':>10} {'ABC':>5}")
print("=" * 90)

for i, row in product_sales.head(10).iterrows():
    rank = product_sales.index.get_loc(i) + 1
    print(f"{rank:<6} {row['商品名']:<40} ¥{row['売上金額']:>13,.0f} "
          f"{row['売上数量']:>9,.0f} {row['ABC分類']:>5}")

---

# 🔮 セクション6: 明日の売上予測と発注推奨

## 需要予測で最適な在庫を実現

In [ ]:
# シンプルな売上予測（移動平均ベース）
print("🔮 明日の売上予測")
print("=" * 60)

# 最近7日間の平均
recent_avg = daily_sales.tail(7)['売上金額'].mean()

# 曜日別トレンド
tomorrow_weekday = (daily_sales['日付'].max() + pd.Timedelta(days=1)).weekday()
weekday_factor = weekday_analysis[
    weekday_analysis['曜日'] == tomorrow_weekday
]['売上金額'].values[0] / weekday_analysis['売上金額'].mean()

# 予測値
predicted_sales = recent_avg * weekday_factor

tomorrow_date = daily_sales['日付'].max() + pd.Timedelta(days=1)
tomorrow_weekday_name = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'][tomorrow_weekday]

print(f"予測日: {tomorrow_date.strftime('%Y-%m-%d')} ({tomorrow_weekday_name})")
print(f"予測売上: ¥{predicted_sales:,.0f}")
print(f"\n📊 予測根拠:")
print(f"  - 直近7日平均: ¥{recent_avg:,.0f}")
print(f"  - {tomorrow_weekday_name}曜日補正: {weekday_factor:.2f}倍")

# 信頼区間
std_dev = daily_sales.tail(7)['売上金額'].std()
lower_bound = predicted_sales - std_dev
upper_bound = predicted_sales + std_dev

print(f"\n📈 予測範囲（±1σ）:")
print(f"  下限: ¥{lower_bound:,.0f}")
print(f"  予測: ¥{predicted_sales:,.0f}")
print(f"  上限: ¥{upper_bound:,.0f}")

# 発注推奨
print("\n📦 発注推奨アクション")
print("=" * 60)

if weekday_factor > 1.1:
    print("✅ 明日は売上増加が見込まれます")
    print("   → Aランク商品の在庫を通常より10-20%増やす")
    print("   → 人気商品の欠品に注意")
elif weekday_factor < 0.9:
    print("⚠️ 明日は売上減少が見込まれます")
    print("   → 生鮮品の発注を控えめに")
    print("   → 賞味期限の短い商品に注意")
else:
    print("📊 明日は平均的な売上が見込まれます")
    print("   → 通常通りの発注でOK")

# TOP10商品の推奨発注数
print("\n📋 TOP10商品の推奨発注数（明日分）")
print("=" * 70)
print(f"{'商品名':<35} {'1日平均':>12} {'明日推奨':>12}")
print("=" * 70)

for _, row in product_sales.head(10).iterrows():
    days_count = (daily_sales['日付'].max() - daily_sales['日付'].min()).days + 1
    avg_qty = row['売上数量'] / days_count
    recommended_qty = int(avg_qty * weekday_factor * 1.1)  # 10%の安全在庫
    
    print(f"{row['商品名']:<35} {avg_qty:>11.1f} {recommended_qty:>11}個")

---

# 🚨 セクション7: アラート・異常検知

## 問題の早期発見で損失を最小化

In [ ]:
# 異常検知
print("🚨 異常検知アラート")
print("=" * 70)

alerts = []

# 1. 売上急落アラート
recent_7days = daily_sales.tail(7)['売上金額'].mean()
previous_7days = daily_sales.tail(14).head(7)['売上金額'].mean()

if recent_7days < previous_7days * 0.9:
    alerts.append({
        'レベル': '🔴 高',
        'タイプ': '売上急落',
        '内容': f'直近7日間の売上が前週比{((recent_7days/previous_7days - 1) * 100):.1f}%減少',
        '対策': '売上減少要因の特定（商品、時間帯、客層など）'
    })

# 2. 前年比大幅減アラート
recent_yoy = daily_sales.tail(7)['前年比'].mean()
if recent_yoy < -10:
    alerts.append({
        'レベル': '🟡 中',
        'タイプ': '前年比悪化',
        '内容': f'直近7日間の前年比が{recent_yoy:.1f}%',
        '対策': '前年の施策・商品構成を確認し、良かった点を再現'
    })

# 3. 客単価低下アラート
recent_avg_spend = daily_sales.tail(7)['客単価'].mean()
overall_avg_spend = daily_sales['客単価'].mean()

if recent_avg_spend < overall_avg_spend * 0.95:
    alerts.append({
        'レベル': '🟡 中',
        'タイプ': '客単価低下',
        '内容': f'直近の客単価¥{recent_avg_spend:.0f}が平均¥{overall_avg_spend:.0f}より低い',
        '対策': 'セット販売、まとめ買い提案、高単価商品の推奨販売'
    })

# 4. 売上変動大アラート
sales_std = daily_sales.tail(14)['売上金額'].std()
sales_mean = daily_sales.tail(14)['売上金額'].mean()
cv = sales_std / sales_mean  # 変動係数

if cv > 0.2:
    alerts.append({
        'レベル': '🟢 低',
        'タイプ': '売上変動大',
        '内容': f'売上のばらつきが大きい（変動係数{cv:.2f}）',
        '対策': '安定的な売上を実現するため、曜日別の標準化を検討'
    })

# アラート表示
if alerts:
    for i, alert in enumerate(alerts, 1):
        print(f"\n【アラート {i}】 {alert['レベル']} - {alert['タイプ']}")
        print(f"状況: {alert['内容']}")
        print(f"推奨対策: {alert['対策']}")
        print("-" * 70)
else:
    print("\n✅ 現在、深刻なアラートはありません")
    print("   順調に運営できています！")

print(f"\n📊 総アラート数: {len(alerts)}件")

---

# 📋 セクション8: 今日のアクションチェックリスト

## 店長が今日やるべきこと

In [ ]:
print("\n✅ 今日のアクションチェックリスト")
print("=" * 70)

checklist = []

# 売上確認
checklist.append("□ 今日の売上実績を確認（目標達成度）")
checklist.append("□ 前日比・前年比を確認")

# 発注関連
checklist.append("□ TOP10商品の在庫状況を確認")
checklist.append("□ 明日の予測売上に基づく発注")
checklist.append("□ Aランク商品の欠品防止")

# 改善施策
if recommendations:
    checklist.append(f"□ 優先施策の実施: {recommendations[0]['施策']}")

# アラート対応
if alerts:
    for alert in alerts:
        if alert['レベル'] == '🔴 高':
            checklist.append(f"□ 緊急対応: {alert['内容']}")

# 日常業務
checklist.append("□ 商品陳列の整理整頓")
checklist.append("□ 賞味期限チェック")
checklist.append("□ 清掃状況の確認")
checklist.append("□ スタッフとの情報共有")

# チェックリスト表示
for i, item in enumerate(checklist, 1):
    print(f"{i:2}. {item}")

print("\n" + "=" * 70)
print("💡 このチェックリストを印刷して、毎日確認しましょう！")

---

# 🎓 まとめ: データドリブン店舗運営の5つのポイント

## 1️⃣ **毎日の数字を追う**
- 売上、客数、客単価の3つの指標を毎日確認
- 前日比・前年比で変化をキャッチ

## 2️⃣ **他店と比較する**
- ベンチマークでギャップを認識
- トップ店舗の施策を学ぶ

## 3️⃣ **重点商品に注力**
- ABC分析で商品の優先順位を明確化
- Aランク商品の欠品は絶対に避ける

## 4️⃣ **予測に基づく発注**
- 曜日別・季節別のトレンドを活用
- 過剰在庫と欠品のバランスを取る

## 5️⃣ **PDCAサイクルを回す**
- 施策を実施 → 効果を測定 → 改善
- 小さな改善の積み重ねが大きな成果に

---

## 📞 サポート

このダッシュボードについて質問があれば、いつでもご連絡ください。

**あなたの店舗の成功を応援しています！** 🎉